# Approaching a 3D Hot Jupiter Model With PLUTO
## Summary of Key Attributes of the Setup
### definitions.h
- HD physics module
- Spherical geometry
- Rotating frame
- 3 day rotation period
- Radial coordinate extends to 1.6 times the radius of Jupiter

### pluto.ini
**Grid and solver**
- X1-grid (radius): 2 coordinate patches, each with with uniform spacing
    - 0.01 to 1.0 has 20 elements
    - 1.0 to 1.2 has 10 elements
- X2-grid (elevation): 1 coordinate patch with 36 elements, extending $\pm80^\circ$ above the equator
- X3-grid (azimuth): 1 coordinate patch with 80 elements, wrapping around the globe completely
- CFL = 0.25 with max var 1.1
- hllc solver
- van Leer limiter

**Boundary conditions**
- X1-beg: axisymmetric
- X1-end: userdef
- X2-beg: reflective
- X2-end: reflective
- X3-beg: periodic
- X3-end: periodic

**User parameters**
- ALPHA = 10.0

### init.c
**Body force**
- The gravitational field is given by
$$
g = 
   \begin{cases} 
      -1/R^2           & \quad{\rm for} \quad R > 1 \\
      aR + bR^2 + cR^3 & \quad{\rm for} \quad R < 1 \\
   \end{cases}
$$
Additionally, the rotating frame weakens the effect of gravity in proportion to the cylindrical radius (about the axis of rotation) due to centrifugal force. PLUTO handles this internally for all dynamics, but the centrifugal force needs to be accounted for manually in initial and boundary conditions. This is still a TODO.

**Initial conditions**
- The density field is initialized to a state of hydrostatic equilibrium (TODO: account for centrifugal force)
- All velocities are initially 0
- Angular velocity of rotating frame is $$\Omega_z = \frac{L}{V T_r}=\frac{(1.6*6.9911\times10^9 [cm]) / 1.2}{10^5 [cm/s] \times3*34*3600 [s]}=0.357$$ in code units, where $L$ and $V$ are the unit length and velocity, respectively, and $T$ is the period of rotation. Note that we define $L$ as the planet radius, $R_p$, divided by the end coordinate of the radius grid ($1.2$) so that unit radius ($1.0$) corresponds to the gravitational field transition from the interior of the planet to the exterior, and maximum radius ($1.2$) corresponds to the edge of the atmosphere

**Boundary conditions**
- At the end of the radial coordinate, all velocities are 0
- At the end of the radial coordinate, density is set to $exp\big(\alpha(\frac{1}{R}-1)\big)$ and pressure is set to $\rho/\alpha$ (isothermal)

**Winds (internal boundary condition)**
- Winds can be enabled using an internal boundary condition, which forces a certain $v_{phi}$ profile in the upper 10% of the fluid
    - Exponential profile (`#define WINDS_EXP`): $$exp\big(10(R/R_p - 1)\big) sin(m\theta + b)$$
    - Quadratic profile (`#define WINDS_QUAD`): $$\bigg(\frac{R}{R_p}\bigg)^2sin(m\theta + b)$$
Where $m$ and $b$ are constants used to make the profiles maximum at the equator, and $0$ at the ends of the elevation grid

# Analysis of Results

In [1]:
import os
import sys
import numpy as np

import matplotlib
matplotlib.use('agg')
import imageio

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyPLUTO as pp

from __future__ import print_function

# Uncomment this to display plots in this notebook. Not recommended
# for large data sets (e.g. more than 10 frames).
# Note that all plots will be saved to the working directory anyway.
# %matplotlib inline

## Loading Data
Make sure ```path_to_script```, below, is set to the location where this script is running

In [2]:
path_to_script = os.path.join(os.environ['PLUTO_DIR'], 'Work\\')
w_dir = os.path.join(path_to_script, "data")
print("Loading data from", w_dir)

data_dirs = [directory for directory in os.listdir(os.path.join(w_dir)) if "." not in directory]
print("Found data directories: ")
i = 0
for dir in data_dirs:
    print("\t{0}: {1}".format(i, dir))
    i = i + 1

Loading data from D:\PLUTO\Work\data
Found data directories: 
	0: 10t_centrifugal_exp_winds
	1: 10t_centrifugal_exp_winds_old
	2: 10t_centrifugal_no_winds
	3: 10t_centrifugal_no_winds_old
	4: 3t_centrifugal_no_winds
	5: 3t_centrifugal_no_winds_old
	6: 3t_with_centrifugal
	7: centrifugal_exp_winds_tau2p8
	8: centrifugal_exp_winds_tau5p6
	9: centrifugal_exp_winds_tau8p4
	10: centrifugal_exp_winds_tau8p4_old
	11: centrifugal_exp_winds_vr0_vtheta0
	12: centrifugal_no_winds
	13: centrifugal_no_winds_vr0_vtheta0
	14: disk_planet_no_rotation
	15: disk_planet_rotation
	16: mag
	17: mag_15t
	18: mag_3t
	19: res_20u10u_36_40
	20: res_20u10u_36_40_alpha31p3
	21: res_20u10u_36_40_alpha3p13
	22: res_26_37_40
	23: res_26_74_80
	24: rotating
	25: rotating_ic
	26: rotating_ic_3d
	27: rotating_ic_3d_outflow
	28: rotating_ic_6d
	29: rotating_potential
	30: rotating_winds_10t
	31: rotating_winds_25t_outflow_exp
	32: rotating_winds_25t_outflow_quad
	33: rotating_winds_25t_reflective_exp
	34: rotating_wind

Next, we may have a choice between different data sets to load. These have been enumerated above. To change the data set, change the value of `set_idx`, below to one of the other indices.

In [3]:
set_idx = 16
wdir = os.path.join(w_dir, data_dirs[set_idx] + os.sep)
print("Using working directory", wdir)

Using working directory D:\PLUTO\Work\data\mag\


Next, we use pyPLUTO to load the data in ```wdir``` into a ```pload``` object. ```dh[n]``` is the data handle for data.```n```.dbl.

In [4]:
n_frames = len([file for file in os.listdir(wdir) if "data" in file and ".dbl" in file])
dh = list() # List of data handles
for i in range(n_frames):
    d = pp.pload(i, w_dir = wdir)
    dh.append(d)

Reading Data file : D:\PLUTO\Work\data\mag\data.0000.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0001.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0002.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0003.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0004.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0005.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0006.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0007.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0008.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0009.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0010.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0011.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0012.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0013.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0014.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0015.dbl
Reading Data file : D:\PLUTO\Work\data\mag\data.0016.dbl
Reading Data file : D:\PLUTO\Wo

## Animations
Now we define some functions and variables to help with making animations from the data

In [5]:
im_set = []
DEFAULT_FRAME_PERIOD = 0.25
    
def save_plt(fname, w_dir=wdir, image_set=im_set, track=True):
    '''
    Saves plots to the working directory

    Parameters
    ----------
    fname : str
        The name to be given to the image file
    wdir : str
        (Optional) The location where the file is to be saved
    image_set : list
        Holds the file names related to this analysis set
    track : bool
        True if this file should be tracked in the analysis set
    '''
    name = fname + '.png'
    plt.savefig(os.path.join(w_dir, name))
    if track:
        image_set.append(name)


def end_set(frame_period=DEFAULT_FRAME_PERIOD, w_dir=wdir, image_set=im_set, del_frames=True):
    # Make .gif animation of the frames
    images = []
    for filename in image_set:
        images.append(imageio.imread(os.path.join(w_dir, filename)))
    gif_name = os.path.join(w_dir, image_set[-1][:-4] + '.gif')
    print(gif_name)
    imageio.mimsave(gif_name, images, duration=frame_period)
    # Delete individual frames to save space
    if del_frames:
        for filename in image_set:
            os.remove(os.path.join(w_dir, filename))
    del image_set[:]
    
def clear_set(image_set=im_set):
    del image_set[:]

## 2D Plots
We now define a function to help us with plotting, and also perform some one-time plot configurations.

In [9]:
def get_field_qty(field, df, idx, const_elevation=False, const_azimuth=False):
    plot_v = field == 'v_phi' or field == 'v_r' or field =='v_theta'
    plot_b = field == 'b_phi' or field == 'b_r' or field =='b_theta'
    assert(field == 'rho' or plot_v or plot_b), "field argument is invalid"
    assert(const_elevation != const_azimuth), "EITHER the azimuth or elevation may be specified, not both"
    
    # Find the field quantity
    if field == 'rho':
        qty = df.rho
    elif plot_v:
        if field == 'v_r':
            qty = df.vx1
        elif field == 'v_theta':
            qty = df.vx2
        elif field == 'v_phi':
            qty = df.vx3
    elif plot_b:
        if field == 'b_r':
            qty = df.bx1
        elif field == 'b_theta':
            qty = df.bx2
        elif field == 'b_phi':
            qty = df.bx3

    # Get the spatial slice
    if const_azimuth:
        qty = qty[:,:,idx].T
    elif const_elevation:
        qty = qty[:,idx,:].T
    
    return qty;


# TODO(Tyler): instead of simply passing in True or False, we should be able to specify
# the desired elevation angle, or azimuthal angle and then choose the closest value
# that we have on the grid
def plot_2d(
    data,
    const_elevation=False,
    theta=90,
    const_azimuth=False,
    phi=0,
    log=False,
    data_idx=-1,
    field='',
    arrows=False,
    v_min=float('inf'),
    v_max=float('inf')
):
    '''
    Plots scalar and vector field quantities.
    
    Parameters
    ----------
    data : list of pyPLUTO.pload
        List of data frames
    const_elevation : bool
        If True, plots the density as a function of radius and elevation
        Exclusive to const_azimuth
    theta : double
        The elevation (latitude) to fix, if applicable. Given in degrees
    phi : double
        The longitude to fix, if applicable. Given in degrees
    const_azimuth : double
        If True, plots the density as a function of radius and azimuth
        Exclusive to const_elevation
    log : bool
        (Optional) Plots the logarithm for scalar fields (base 10)
    data_idx : int
        (Optional) Causes only the specified frame to be plotted
    field : str
        The field quantity to plot
            - "rho" for density
            - "v_phi" for azimuthal velocity
            - "v_r" for radial velocity
            - "v_theta" for meridional velocity
    arrows : bool
        Switch to plot velocity vectors
    v_min : float
        Specifies what the colorbar's min color should be
    v_max : float
        Specifies what the colorbar's max color should be
    '''
    
    if const_elevation == False and const_azimuth == False:
        const_elevation = True
    
    # Sanity check on inputs
    plot_v = field == 'v_phi' or field == 'v_r' or field =='v_theta'
    plot_b = field == 'b_phi' or field == 'b_r' or field =='b_theta'
    assert(field == '' or plot_v or plot_b), "field argument is invalid"
    assert(const_elevation != True or const_azimuth != True), "EITHER the azimuth or elevation may be specified, not both"
    assert(not (field == '' and log == True)), "Cannot do log plot if no field quantity is specified"
    assert(data_idx <= len(dh)), "data_idx must be <= len(dh)"
    
    single_only = (data_idx != -1)
    nframes = len(data)
    nrows   = 1 if single_only else np.round(np.sqrt(nframes))
    ncols   = 1 if single_only else np.round(np.sqrt(nframes) + 1)
    theta_idx = (np.abs(dh[0].x2 - theta * np.pi / 180)).argmin()
    phi_idx   = (np.abs(dh[0].x3 - phi * np.pi / 180)).argmin()
    fig = plt.figure()
    for idx in range(nframes):
        if single_only and idx != data_idx:
            continue
        # Set up quantities
        if plot_v:
            U = data[idx].vx1
        elif plot_b:
            U = data[idx].bx1
        X = data[idx].x1
        if const_azimuth:
            Y = 90 - (180 / np.pi) * data[idx].x2
            if plot_v:
                V = -1 * data[idx].vx2
            if plot_b:
                V = -1 * data[idx].bx2
            U = U[:,:,phi_idx].T
            V = V[:,:,phi_idx].T
            annotations = [field, 'X1 (radius)', 'X2 (elevation)']
            qty = get_field_qty(field, data[idx], phi_idx, const_azimuth=True)
        elif const_elevation:
            Y = (180 / np.pi) * data[idx].x3
            if plot_v:
                V = data[idx].vx3
            if plot_b:
                V = data[idx].bx3
            U = U[:,theta_idx,:].T
            V = V[:,theta_idx,:].T
            annotations = [field, 'X1 (radius)', 'X3 (azimuth)']
            qty = get_field_qty(field, data[idx], theta_idx, const_elevation=True)
    
        # Plotting
        if single_only:
            plt.subplot(nrows, ncols, 1)
        else:
            plt.subplot(nrows, ncols, idx + 1)
        if field != '':
            if log:
                annotations[0] = "Log10 " + annotations[0]
                qty = np.log10(qty)
            if v_min == float('inf'):
                v_min = np.floor(np.min(qty))
            if v_max == float('inf'):
                v_max = np.ceil(np.max(qty))
            plt.pcolormesh(X, Y, qty, vmin=v_min, vmax=v_max)
            if (single_only or ((idx != 0 and (idx+1) % ncols == 0) or idx == nframes-1)):
                plt.colorbar()
        if arrows:
            plt.quiver(X, Y, U, V)
        annotations[0] = annotations[0] +  ' (time: ' + str(np.round(data[idx].SimTime, 2)) + ')'
        plt.title(annotations[0])
        plt.xlabel(annotations[1])
        if single_only or idx % ncols == 0:
            plt.ylabel(annotations[2])
        if single_only:
            break

    # Save image to working directory
    if const_azimuth:
        fname = "rad_and_theta_vs_" + field
    elif const_elevation:
        fname = "rad_and_phi_vs_" + field
    if arrows:
        if fname[-1] != '_':
            fname = fname + "_"
        fname = fname + "vector"
    fname = fname + "_max" + str(v_max)
    if not single_only:
        save_plt(fname, track=False)
    else:
        fname = fname + "_t" + str(np.round(data[idx].SimTime, 2))
        save_plt(fname)
    plt.close();


# Inches
width = 20
height = 25

# One-time plot configuration
plt.rcParams['figure.figsize'] = [width, height]
plt.tight_layout()
plt.subplots_adjust(hspace=0.5)

We now fix the azimuth, and plot how the density varies with radius and elevation. The code below will either plot this relationship for only the first time snapshot, or for all time snapshots depending on whether `only_plot_first_frame`, below, is `True` or `False`.

# Vector Plots

## Analysis Parameters

In [10]:
# Plot a few specific frames. Note that -1 will plot all frames
#frames = [-1, 0, 5, 10, 15]
frames = range(len(dh))

# Scale to these velocities for each frame
v_analyze = [0.01, 1]

# Scale to these magnetic fields for each frame
b_analyze = [1e-4]

## Velocities on an elevation slice

In [11]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='v_r', arrows=True, v_min=-1*vmax, v_max=vmax)
    end_set()

D:\PLUTO\Work\data\mag\rad_and_theta_vs_v_r_vector_max0.01_t30.0.gif
D:\PLUTO\Work\data\mag\rad_and_theta_vs_v_r_vector_max1_t30.0.gif


In [12]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='v_theta', arrows=True, v_min=-1*vmax, v_max=vmax)
    end_set()

D:\PLUTO\Work\data\mag\rad_and_theta_vs_v_theta_vector_max0.01_t30.0.gif
D:\PLUTO\Work\data\mag\rad_and_theta_vs_v_theta_vector_max1_t30.0.gif


## Zonal velocities at equator

In [13]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        plot_2d(dh, const_elevation=True, theta=90, data_idx=idx, field='v_phi', arrows=True, v_min=-1*vmax, v_max=vmax)
    end_set()

D:\PLUTO\Work\data\mag\rad_and_phi_vs_v_phi_vector_max0.01_t30.0.gif
D:\PLUTO\Work\data\mag\rad_and_phi_vs_v_phi_vector_max1_t30.0.gif


## Magnetic field on an elevation slice

In [14]:
clear_set()
for bmax in b_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='b_r', arrows=True, v_min=-1*bmax, v_max=bmax)
    end_set()

D:\PLUTO\Work\data\mag\rad_and_theta_vs_b_r_vector_max0.0001_t30.0.gif


In [15]:
clear_set()
for bmax in b_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='b_theta', arrows=True, v_min=-1*bmax, v_max=bmax)
    end_set()

D:\PLUTO\Work\data\mag\rad_and_theta_vs_b_theta_vector_max0.0001_t30.0.gif


## Zonal magnetic field at equator

In [16]:
clear_set()
for bmax in b_analyze:
    for idx in frames:
        plot_2d(dh, const_elevation=True, theta=90, data_idx=idx, field='b_phi', arrows=True, v_min=-1*bmax, v_max=bmax)
    end_set()

D:\PLUTO\Work\data\mag\rad_and_phi_vs_b_phi_vector_max0.0001_t30.0.gif


## Density

In [ ]:
idx = -1 # Make this negative, or omit, to plot for all time frames
plot_2d(dh, const_azimuth=True, log=True, data_idx=idx, field='rho')
save_plt('rho_vs_rad_vs_lat')

## Radial Density Profile
Here we will plot how the density varies with radius, when the elevation and azimuth are fixed. We will define this as a general-purpose function so that we can reuse it.

In [ ]:
def subplot_radius_vs_density_2d(data, theta, phi, log=False, first_only=False):
    '''
    Plots density profiles as a function of the radius for fixed theta and phi, for
    all time steps.
    
    Parameters
    ----------
    data : list of pyPLUTO.pload
        List of data frames
    theta : double
        The desired elevation angle (spherical coordinates)
    phi : double
        The desired azimuthal angle
    log : bool
        (Optional) Plots the logarithm of the density (base 10)
    first_only : bool
        (Optional) Causes only the first frame to be plotted
        
    Note that the values specified for theta and/or phi may not exist on the grid since
    it is discrete. The closest value will be used in such cases.
    '''
    
    theta_idx = (np.abs(dh[0].x2 - theta * np.pi / 180)).argmin()
    phi_idx = (np.abs(dh[0].x3 - phi * np.pi / 180)).argmin()
    
    nframes = 1 if first_only else len(data)
    nrows   = 1 if first_only else np.round(np.sqrt(nframes))
    ncols   = 1 if first_only else np.round(np.sqrt(nframes) + 1)
    for idx in range(nframes):
        plt.subplot(nrows, ncols, idx + 1)
        qty = np.log10(dh[idx].rho[:, theta_idx, phi_idx])
        plt.scatter(dh[idx].x1, qty)
        plt.title('Time: ' + str(np.round(dh[idx].SimTime, 2)) +
                  ' (theta = ' + str(np.round(dh[idx].x2[theta_idx] * 180 / np.pi, decimals=1)) +
                  ', phi = ' + str(np.round(dh[idx].x3[phi_idx] * 180 / np.pi, decimals=1)) + ')'
        )
        plt.xlabel('X1 (radius)')
        plt.ylabel('Log 10 Density')

### North Pole

In [ ]:
subplot_radius_vs_density_2d(dh, 0, 0, first_only=False);
save_plt('rho_vs_rad_north_pole_profile')

### Equator

In [ ]:
subplot_radius_vs_density_2d(dh, 90, 0, first_only=False);
save_plt('rho_vs_rad_equator_profile')

### South Pole

In [ ]:
subplot_radius_vs_density_2d(dh, 180, 0, first_only=False);
save_plt('rho_vs_rad_south_pole_profile')

## 3D Plots
First we will define some functions to help us with plotting in 3D. The scatter plots only work nicely with cartesian coordinates so we will have to convert from spherical.

In [ ]:
def sph2cart(r, theta, phi):
    '''
    Converts the inputted spherical coordinates to cartesian coordinates
    
    Parameters
    ----------
    r : double
        The radius
    theta : double
        The altitude
    phi : double
        The azimuth
    '''
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z


def sph2cart(r, theta, phi):
    '''
    Converts the inputted spherical coordinates to cartesian coordinates
    
    Parameters
    ----------
    r : double
        The radius
    theta : double
        The altitude
    phi : double
        The azimuth
    '''
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z


def pltSphData3D(d, qty, axes, r=-1, theta=-1, phi=-1, silent=False, f=-1,
    log=False, v_min=float('inf'), v_max=float('inf'), size=20
):
    '''
    Displays a 3D scatter plot of the specified field quantity with one of the
    coordinates held constant.
    
    Parameters
    ----------
    d : pyPLUTO.pload
        Interface to data object
    qty : str
        Name of field quantity (e.g. 'rho' or 'prs')
    axes : matplotlib.axes.SubplotBase
        Plotting interface
    r, theta, phi : int
        (Optional) Fixes a grid index for the radius, elevation, or azimuth. This
        quantity will be kept constant throughout the plot. Only one may be set at
        a time. Defaults to a plot of constant radius.
    silent : bool
        Prints info messages when True, otherwise no messages are printed
    log : bool
        (Optional) Plots the logarithm of the field quantity (base 10)
    v_min : float
        Specifies the min value for the color scale
    v_max : float
        Specifies the max value for the color scale
    size : float
        Specifies point size to pass into the plotter
    
    Example
    -------
    # This will use the pload object `dh` to plot the density for a constant radius.
    The radius setting of 20 fixes the radius at dh.x1[20].
    pltSphData3D(dh, 'rho', r=20)
    '''
    
    # We default to a constant radius (specifically, the innermost radius)
    if r == -1 and phi == -1 and theta == -1:
        r = 0
    
    # Sanity checks on inputs
    assert(
        (r >= 0  and phi == -1 and theta == -1) or
        (r == -1 and phi >= 0  and theta == -1) or
        (r == -1 and phi == -1 and theta >= 0 )
    ), "Only one of r, phi, and theta can be specified"
    
    assert(qty == 'rho' or qty == 'prs' or qty == 'v_r' or qty == 'v_theta' or qty == 'v_phi'), "Invalid field quantity"
    
    # Coordinates
    #    x1 = radial
    #    x2 = latitudinal (theta)
    #    x3 = longitudinal (azimuthal, phi)
    x, y, z, c = [], [], [], []
    
    num_r = d.x1.shape[0]
    num_theta = d.x2.shape[0]
    num_phi = d.x3.shape[0]
    
    # Select either density or pressure based on the string passed in
    if qty == 'rho':
        field = d.rho
    elif qty == 'prs':
        field = d.prs
    elif qty == 'v_r':
        field = d.vx1
    elif qty == 'v_theta':
        field = d.vx2
    elif qty == 'v_phi':
        field = d.vx3

    if log:
        field = np.log(field)
    
    # Plot the quantity
    if(r >= 0):
        assert(r <= num_r - 1), "r is too large (must be <= {0})".format(num_r - 1)
        r_l = d.x1[r]
        if not silent:
            print("Plotting sphere of radius {0}".format(r_l))
        for i in range(num_theta):
            theta_l = d.x2[i]
            for j in range(num_phi):
                phi_l = d.x3[j]
                x_p, y_p, z_p = sph2cart(r_l, theta_l, phi_l)
                x.append(x_p)
                y.append(y_p)
                z.append(z_p)
                c.append(field[r,i,j])
    elif(theta >= 0):
        assert(theta <= num_theta - 1), "theta is too large (must be <= {0})".format(num_theta - 1)
        theta_l = d.x2[theta]
        if not silent:
            print("Plotting cone with elevation angle {0} deg".format(theta_l * 180 / np.pi))
        for i in range(num_r):
            r_l = d.x1[i]
            for j in range(num_phi):
                phi_l = d.x3[j]
                x_p, y_p, z_p = sph2cart(r_l, theta_l, phi_l)
                x.append(x_p)
                y.append(y_p)
                z.append(z_p)
                c.append(field[i,theta,j])
    else:
        assert(phi <= num_phi - 1), "phi is too large (must be <= {0})".format(num_phi - 1)
        phi_l = d.x3[phi]
        if not silent:
            print("Plotting disk with azimuth {0} deg".format(phi_l * 180 / np.pi))
        for i in range(num_r):
            r_l = d.x1[i]
            for j in range(num_theta):
                theta_l = d.x2[j]
                x_p, y_p, z_p = sph2cart(r_l, theta_l, phi_l)
                x.append(x_p)
                y.append(y_p)
                z.append(z_p)
                c.append(field[i,j,phi])
    
    plt.xlabel('x')
    plt.ylabel('y')
    
    # Update the plot axes, but don't make them smaller
    axes.set_xlim3d(
        min(axes.get_xlim3d()[0], min(x)),
        max(axes.get_xlim3d()[1], max(x))
    )
    axes.set_ylim3d(
        min(axes.get_ylim3d()[0], min(y)),
        max(axes.get_ylim3d()[1], max(y))
    )
    axes.set_zlim3d(
        min(axes.get_zlim3d()[0], min(z)),
        max(axes.get_zlim3d()[1], max(z))
    )
    
    # Compute the max and min values used for the color
    if v_min == float('inf'):
        v_min = np.floor(np.min(field))
    if v_max == float('inf'):
        v_max = np.ceil(np.max(field))
    axes.scatter3D(x, y, z, c=c, vmin=v_min, vmax=v_max, s=size)

### Constant Azimuth

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=40, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=79, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_const_azimuth_vtheta' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
    end_set()

### Combined Plots

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, phi=10, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, phi=50, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, theta=0, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, theta=35, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, r=1, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_azimuth_and_elevation_vphi' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
    end_set()

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=10, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=50, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, theta=0, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, theta=35, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, r=1, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_azimuth_and_elevation_vtheta' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
    end_set()

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, phi=10, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, phi=50, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, theta=0, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, theta=35, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, r=1, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_azimuth_and_elevation_vr' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
    end_set()